<a href="https://colab.research.google.com/github/XuyangAbert/DeepALCS/blob/main/DeepAL3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
from scipy.spatial.distance import pdist,squareform
from sklearn.preprocessing import MinMaxScaler
# from featureselection import SFSFC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import f1_score,precision_score,auc, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import time
import pandas as pd
from math import exp
import numpy.matlib
# import tensorflow.compat.v1 as tf
import tensorflow as tf
# from keras.applications.resnet50     import ResNet50
# from keras.applications.vgg16        import VGG16
# from keras.applications.vgg19        import VGG19
from keras.models import Model
from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation
# from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Dense, AveragePooling2D, GlobalAveragePooling2D, MaxPool2D
from keras.applications.inception_v3 import preprocess_input as incv3_preprocess_input
# from keras.applications.resnet50     import preprocess_input as resnet50_preprocess_input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10, cifar100,fashion_mnist
# from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.vgg16        import preprocess_input as vgg16_preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
# from livelossplot.inputs.keras import PlotLossesCallback

In [ ]:
!pip install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 6.0 MB/s 
     |████████████████████████████████| 160 kB 65.7 MB/s 
     |████████████████████████████████| 192.9 MB 80 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=84937b4690f669d8d5fb62612c2100e933ffe9d0591b0fce2cbe728257fefc5e
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
import tpot

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from tpot import TPOTClassifier
# define dataset
X, y = make_classification(n_samples=100, n_features=10, n_informative=5, n_redundant=5, random_state=1)
# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search
model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='accuracy', verbosity=2, random_state=1, n_jobs=-1)
# perform the search
model.fit(X, y)
# export the best model
model.export('tpot_best_model.py')

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9399999999999998

Generation 2 - Current best internal CV score: 0.9399999999999998

Generation 3 - Current best internal CV score: 0.9399999999999998

Generation 4 - Current best internal CV score: 0.9466666666666667

Generation 5 - Current best internal CV score: 0.9466666666666667

Best pipeline: MLPClassifier(LinearSVC(input_matrix, C=5.0, dual=True, loss=squared_hinge, penalty=l2, tol=0.0001), alpha=0.001, learning_rate_init=0.1)


In [ ]:
def Pre_Data(data):
    [N, L] = np.shape(data)
    NewData = np.empty((N, L))
    for i in range(L):
        Temp = data[:, i]
        NewData[:, i] = Temp
    return NewData


def ParamSpe(data):
    Buffersize = 1000
    PreStd = []
    P_Summary = []
    PFS = []
    T = round(np.shape(data)[0] / Buffersize)
    return Buffersize, P_Summary, T, PFS, PreStd


def FeatureType(data):
    [N, dim] = np.shape(data)
    th = round(N ** 0.5)
    F_cont = []
    F_disc = []
    F_zero = []
    for j in range(dim):
        temp_unique = np.unique(data[:, j])
        if len(temp_unique) > th:
            F_cont.append(j)
        elif len(temp_unique) > 1:
            F_disc.append(j)
        if len(temp_unique) == 1:
            F_zero.append(j)
    return F_cont, F_disc, F_zero


def Distance_Cal(data):
    D = pdist(data)
    Dist = squareform(D)
    return Dist


def Fitness_Cal(sample, pop, stdData, gamma):
    Ns = np.shape(sample)[0]
    Np = np.shape(pop)[0]
    Newsample = np.concatenate([sample, pop])
    Dist = Distance_Cal(Newsample)
    fitness = []
    for i in range(Np):
        distArray = np.power(Dist[i + Ns, 0:Ns], 2)
        temp = np.power(np.exp(-distArray / stdData), gamma)
        fitness.append(np.sum(temp))
    return fitness


def fitness_update(P_Summary, Current, fitness, PreStd, gamma, stdData):
    [N, dim] = np.shape(Current)
    t_I = len(PreStd)
    NewFit = fitness
    if len(P_Summary) > 0:
        PreFit = P_Summary[:, dim]
        PreP = P_Summary[:, 0:dim]
        OldStd = PreStd[t_I - 1]
        for i in range(N):
            fitin = 0
            for j in range(np.shape(PreP)[0]):
                if np.linalg.norm(Current[i][:] - PreP[j][:]) < 0.01:
                    fitin = PreFit[j]
                    break
                else:
                    d = np.linalg.norm(Current[i][:] - PreP[j][:])
                    fitin += (exp(-d ** 2 / stdData) ** gamma) * (PreFit[j] ** (OldStd / stdData))
            NewFit[i] = fitness[i] + fitin
    return NewFit


def PopInitial(sample, PreMu, PreStd, Buffersize):
    [N, L] = np.shape(sample)
    pop_Size = round(1 * N)
    # Compute the statistics of the current data chunk
    minLimit = np.min(sample, axis=0)
    meanData = np.mean(sample, axis=0)
    maxLimit = np.max(sample, axis=0)
    # Update the statistics of the data stream
    meanData = UpdateMean(PreMu, meanData, Buffersize)
    PreMu.append(meanData)
    # Compute the standard deviation of the current data chunk
    MD = np.matlib.repmat(meanData, N, 1)
    tempSum = np.sum(np.sum((MD - sample) ** 2, axis=1))
    stdData = tempSum / N
    # Update the standard deviation of the data stream
    stdData = StdUpdate(stdData, PreStd, Buffersize)
    # Randonmly Initialize the population indices from the data chunk
    pop_Index = np.arange(0, N)
    pop = sample[pop_Index, :]
    # Calculate the initial niche radius
    radius = numpy.linalg.norm((maxLimit - minLimit)) * 0.4  # 0.6

    return [stdData, pop_Index, pop, radius, PreMu, PreStd]


def UpdateMean(PreMy, meanData, BufferSize):
    # Num of the processed data chunk
    t_P = len(PreMu)
    # Update the mean of the data stream as new data chunk arrives
    if t_P == 0:
        newMu = meanData
    else:
        oldMu = PreMu[t_P - 1][:]
        newMu = (meanData + oldMu * t_P) / (t_P + 1)
    return newMu


def StdUpdate(Std, PreStd, BufferSize):
    # Num of the processed data chunk
    t_P = len(PreStd)
    # Update the variance of the data stream as new data chunk arrives
    if t_P == 0:
        newStd = Std
    else:
        oldStd = PreStd[t_P - 1]
        newStd = (Std + oldStd * t_P) / (t_P + 1)
    return newStd


# ------------------------Parameter Estimation----------------------------#
def CCA(sample, stdData, Dist):
    m = 1
    gamma = 5
    ep = 0.998
    N = np.shape(sample)[0]
    while 1:
        den1 = []
        den2 = []
        for i in range(N - 1):
            Diff = np.power(Dist[i, :], 2)
            temp1 = np.power(np.exp(-Diff / stdData), gamma * m)
            temp2 = np.power(np.exp(-Diff / stdData), gamma * (m + 1))
            den1.append(np.sum(temp1))
            den2.append(np.sum(temp2))

        y = np.corrcoef(den1, den2)[0, 1]

        if y > ep:
            break
        m = m + 1
    return m * gamma


def DCCA(sample, stdData, P_Summary, gamma, dim):
    P_Center = P_Summary[:, 0:dim]
    P_F = P_Summary[:, dim]
    gam1 = gamma
    N1 = np.shape(sample)[0]
    N2 = np.shape(P_Center)[0]
    ep = 0.998
    N = N1 + N2
    temp = np.concatenate([sample, P_Center], axis=0)
    Dist = Distance_Cal(temp)
    while 1:
        gam2 = gam1 + 5
        den1 = []
        den2 = []
        for i in range(N):
            Diff = np.power(Dist[i, 0:N1], 2)
            temp1 = np.power(np.exp(-Diff / stdData), gam1)
            temp2 = np.power(np.exp(-Diff / stdData), gam2)
            sum1 = np.sum(temp1)
            sum2 = np.sum(temp2)
            if i < N1:
                T1 = 0
                T2 = 0
                for j in range(N2):
                    T1 += P_F[j] ** (gam1 / gamma)
                    T2 += P_F[j] ** (gam2 / gamma)
                s1 = sum1 + T1
                s2 = sum2 + T2
            #                s1 = sum1**(gam1/gamma) + T1
            #                s2 = sum2**(gam2/gamma) + T2
            else:
                #                s1 = sum1**(gam1/gamma) + P_F[i-N1]**(gam1/gamma)
                #                s2 = sum2**(gam2/gamma) + P_F[i-N1]**(gam2/gamma)
                s1 = sum1 + P_F[i - N1] ** (gam1 / gamma)
                s2 = sum2 + P_F[i - N1] ** (gam2 / gamma)
            den1.append(s1)
            den2.append(s2)
        y = np.corrcoef(den1, den2)[0, 1]
        if y > ep:
            break
        gam1 = gam2
    return gam1


def TPC_Search(Dist, Pop_Index, Pop, radius, fitness):
    # Extract the size of the population
    [N, dim] = np.shape(Pop)
    P = []  # Initialize the Peak Vector
    P_fitness = []
    # i = 1
    marked = []
    co = []
    OriginalIndice = Pop_Index
    while 1:
        # -------------Search for the local maximum-----------------#
        SortIndice = np.argsort(fitness)
        NewIndice = SortIndice[::-1]

        Pop = Pop[NewIndice, :]
        fitness = fitness[NewIndice]
        OriginalIndice = OriginalIndice[NewIndice]

        P.append(Pop[0, :])

        P_fitness.append(fitness[0])
        P_Indice = OriginalIndice[0]

        Ind = AssigntoPeaks(Pop, Pop_Index, P, P_Indice, marked, radius, Dist)

        marked.append(Ind)
        marked.append(NewIndice[0])

        if not Ind:
            Ind = [NewIndice[0]]

        co.append(len(Ind))
        TempFit = fitness
        sum1 = 0
        for j in range(len(Ind)):
            sum1 += fitness[np.where(OriginalIndice == Ind[j])]
        for th in range(len(Ind)):
            TempFit[np.where(OriginalIndice == Ind[th])] = fitness[np.where(OriginalIndice == Ind[th])] / sum1
        fitness = TempFit
        if np.sum(co) >= N:
            P = np.asarray(P)
            P_fitness = np.asarray(P_fitness)
            break

    return P, P_fitness


def MergeInChunk(P, P_fitness, sample, gamma, stdData):
    """Perform the Merge of TPCs witnin each data chunk
    """
    # Num of TPCs
    [Nc, dim] = np.shape(P)
    NewP = []
    NewP_fitness = []
    marked = []
    unmarked = []
    Com = []

    # Num of TPCs
    Nc = np.shape(P)[0]
    for i in range(Nc):
        MinDist = np.inf
        MinIndice = 100000
        if i not in marked:
            for j in range(Nc):
                if j != i and j not in marked:
                    d = np.linalg.norm(P[j, :] - P[i, :])
                    if d < MinDist:
                        MinDist = d
                        MinIndice = j
            if MinIndice <= Nc:
                MinIndice = int(MinIndice)
                Merge = True
                Neighbor = P[MinIndice][:]
                X = (Neighbor + P[i, :]) / 2

                X = np.reshape(X, (1, np.shape(P)[1]))

                fitX = Fitness_Cal(sample, X, stdData, gamma)
                fitP = P_fitness[i]
                fitN = P_fitness[MinIndice]
                if fitX < 0.85 * min(fitN, fitP):
                    Merge = False
                if Merge:
                    Com.append([i, MinIndice])
                    marked.append(MinIndice)
                    marked.append(i)
                else:
                    unmarked.append(i)
    Com = np.asarray(Com)
    # Number of Possible Merges:
    Nm = np.shape(Com)[0]
    for k in range(Nm):
        if P_fitness[Com[k, 0]] >= P_fitness[Com[k, 1]]:
            NewP.append(P[Com[k, 0], :])
            NewP_fitness.append(P_fitness[Com[k, 0]])
        else:
            NewP.append(P[Com[k, 1], :])
            NewP_fitness.append(P_fitness[Com[k, 1]])
    # Add Unmerged TPCs to the NewP
    for n in range(Nc):
        if n not in Com:
            NewP.append(P[n, :])
            NewP_fitness.append(P_fitness[n])
    NewP = np.asarray(NewP)
    NewP_fitness = np.asarray(NewP_fitness)
    return NewP, NewP_fitness


def MergeOnline(P, P_fitness, P_summary, PreStd, sample, gamma, stdData):
    """Perform the Merge of Clusters Between Historical and New Clusters
    """
    # Num of TPCs
    [Nc, dim] = np.shape(P)
    NewP = []
    NewP_fitness = []
    marked = []
    unmarked = []
    Com = []

    for i in range(Nc):
        MinDist = np.inf
        MinIndice = 100000
        if i not in marked:
            for j in range(Nc):
                if j != i and j not in marked:
                    d = np.linalg.norm(P[j, :] - P[i, :])
                    if d < MinDist:
                        MinDist = d
                        MinIndice = j
            if MinIndice < Nc:

                #                MinIndice = int(MinIndice)
                Merge = True
                Neighbor = P[MinIndice][:]
                X = (Neighbor + P[i][:]) / 2
                X = np.reshape(X, (1, np.shape(P)[1]))
                RfitX = Fitness_Cal(sample, X, stdData, gamma)
                fitX = fitness_update(P_Summary, X, RfitX, PreStd, gamma, stdData)
                fitP = P_fitness[i]
                fitN = P_fitness[MinIndice]
                if fitX < 0.85 * min(fitN, fitP):
                    Merge = False
                if Merge:
                    Com.append([i, MinIndice])
                    marked.append(MinIndice)
                    marked.append(i)
                else:
                    unmarked.append(i)
    Com = np.asarray(Com)
    # Number of Possible Merges:
    Nm = np.shape(Com)[0]
    for k in range(Nm):
        if P_fitness[Com[k, 0]] >= P_fitness[Com[k, 1]]:
            NewP.append(P[Com[k, 0]][:])
            NewP_fitness.append(P_fitness[Com[k, 0]])
        else:
            NewP.append(P[Com[k, 1]][:])
            NewP_fitness.append(P_fitness[Com[k, 1]])
    # Add Unmerged TPCs to the NewP
    for n in range(Nc):
        if n not in Com:
            NewP.append(P[n][:])
            NewP_fitness.append(P_fitness[n])
    NewP = np.asarray(NewP)
    NewP_fitness = np.asarray(NewP_fitness)
    return NewP, NewP_fitness


def CE_InChunk(sample, P, P_fitness, stdData, gamma):
    while 1:
        HistP = P
        #        HistPF = P_fitness
        P, P_fitness = MergeInChunk(P, P_fitness, sample, gamma, stdData)
        if np.shape(P)[0] == np.shape(HistP)[0]:
            break
    return P, P_fitness


def CE_Online(sample, P_Summary, P, P_fitness, stdData, gamma, PreStd):
    dim = np.shape(P)[1]

    # Concatenate the historical and new clusters together
    PC = np.concatenate([P_Summary[:, 0:dim], P])
    RPF = Fitness_Cal(sample, PC, stdData, gamma)
    PF = fitness_update(P_Summary, PC, RPF, PreStd, gamma, stdData)

    while 1:
        HistPC = PC
        #        HistPF = PF
        PC, PF = MergeOnline(PC, PF, P_Summary, PreStd, sample, gamma, stdData)
        RPF = Fitness_Cal(sample, PC, stdData, gamma)
        PF = fitness_update(P_Summary, PC, RPF, PreStd, gamma, stdData)
        if np.shape(PC)[0] == np.shape(HistPC)[0]:
            break
    return PC, PF


def ClusterValidation(sample, P):
    while 1:
        NewP = []
        PreP = P
        [R_d, RIndice] = Cluster_Assign(sample, P)

        for i in range(np.shape(P)[0]):
            Temp = np.where(RIndice == i)
            Temp = np.asarray(Temp)
            if np.shape(Temp)[1] > 2:
                NewP.append(P[i][:])
        P = NewP
        if np.shape(P)[0] == np.shape(PreP)[0]:
            break
    return np.asarray(P)


def ClusterSummary(P, PF, P_Summary, sample):
    dim = np.shape(sample)[1]
    Rp = AverageDist(P, P_Summary, sample, dim)
    P = np.asarray(P)
    PF = [PF]

    PF = np.asarray(PF)
    Rp = np.reshape(Rp, (np.shape(P)[0], 1))
    PCluster = np.concatenate([P, PF.T], axis=1)
    PCluster = np.concatenate([PCluster, Rp], axis=1)

    P_Summary = PCluster

    return P_Summary


def StoreInf(PF, PFS, PreStd, stdData):
    PreStd.append(stdData)
    PFS.append(PF)
    return PreStd, PFS


# --------------------Cluster Radius Computation and Update--------------------#
def AverageDist(P, P_Summary, sample, dim):
    P = P
    # Obtain the assignment of clusters
    [distance, indices] = Cluster_Assign(sample, P)
    rad1 = []
    # if the summary of clusters is not empty
    if len(P_Summary) > 0:

        PreP = P_Summary[:, 0:dim]  # Hstorical Cluster Center vector
        PreR = P_Summary[:, dim + 1]
        for i in range(np.shape(P)[0]):
            if np.shape(np.where(indices == i))[1] > 1:
                SumD1 = 0
                Count1 = 0
                for j in range(np.shape(sample)[0]):
                    if indices[j] == i:
                        SumD1 += distance[j]
                        Count1 += 1
                rad1.append(SumD1 / Count1)
            else:
                C_d = []
                for k in range(np.shape(PreP)[0]):
                    C_d.append(np.linalg.norm(P[i][:] - PreP[k][:]))
                CI = np.argmin(C_d)
                rad1.append(PreR[CI])
    elif not P_Summary:
        for i in range(np.shape(P)[0]):
            SumD1 = 0
            Count1 = 0
            for j in range(np.shape(sample)[0]):
                if indices[j] == i:
                    SumD1 += distance[j]
                    Count1 += 1
            rad1.append(SumD1 / Count1)
    return np.asarray(rad1)


def AssigntoPeaks(pop, pop_index, P, P_I, marked, radius, Dist):
    temp = []
    [N, L] = np.shape(pop)
    for i in range(N):
        distance = Dist[i, P_I]
        if not np.any(marked == pop_index[i]):
            if distance < radius:
                temp.append(pop_index[i])
    indices = temp
    return indices


def Cluster_Assign(sample, P):
    # Number of samples
    N = np.shape(sample)[0]
    # Number of Clusters at t
    Np = np.shape(P)[0]
    MinDist = []
    MinIndice = []
    for i in range(N):
        d = []
        for j in range(Np):
            d.append(np.linalg.norm(sample[i][:] - P[j][:]))
        if len(d) <= 1:
            tempD = d
            tempI = 0
        else:
            tempD = np.min(d)
            tempI = np.argmin(d)

        MinDist.append(tempD)
        MinIndice.append(tempI)
    MinDist = np.asarray(MinDist)
    MinIndice = np.asarray(MinIndice)
    return MinDist, MinIndice
def compute_radius(MinDist,ClusterIndice):
    cluster = np.unique(ClusterIndice)
    nc = len(cluster)
    cluster_rad = []
    for i in range(nc):
        currentcluster = np.where(ClusterIndice==cluster[i])[0]
        cluster_rad.append(np.mean(MinDist[currentcluster]))
    return cluster_rad

**Clustering Procedure using the FPS-clustering**

In [ ]:
#---------------------------Main Function-------------------------#
if __name__ == '__main__':
    start = time.time()
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    # (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    X_train /= 255
    X_test /= 255

    data = X_train.reshape(X_train.shape[0], 3072)
    label = y_train

    dim = np.shape(data)[1]
    [BufferSize,P_Summary,T,PFS,PreStd] = ParamSpe(data)
    T = int(T)
    gammaHist = []
    PFS = []
    PreMu = []

    for t in range(T):
        if t < T-1:
           sample = data[t*BufferSize:(t+1)*BufferSize,:]
        else:
            sample = data[t*BufferSize:np.shape(data)[0]]
        if t==0:
            AccSample = sample
        else:
            AccSample = np.concatenate([AccSample,sample])

        [stdData,pop_index,pop,radius,PreMu,PreStd] = PopInitial(sample,PreMu,PreStd,BufferSize)
        # Initialize the fitness vector
        fitness = np.zeros((len(pop_index),1))
        # Initialize the indices vector
        indices = np.zeros((len(pop_index),1))
        Dist = Distance_Cal(sample)
        if PreStd:
            if PreStd[len(PreStd)-1] > stdData:
                P = P_Summary[:,0:dim]
                localFit = Fitness_Cal(sample,P,stdData,gamma)
                PF = fitness_update(P_Summary,P,localFit,PreStd,gamma,stdData)
                P_Summary = ClusterSummary(P,PF,P_Summary,sample)
                PFS.append(PF)
                PreStd.append(stdData)
                clustercenter = P
                [Assign,clusterindex] = Cluster_Assign(AccSample,P)
                continue
        else:
            gamma = CCA(sample,stdData,Dist)
        gammaHist.append(gamma)
        fitness = Fitness_Cal(sample,pop,stdData,gamma)
        fitness = np.array(fitness)
        P, P_fitness = TPC_Search(Dist,pop_index,pop,radius,fitness)
        P, P_fitness = CE_InChunk(sample,P,P_fitness,stdData,gamma)
        P_fitness = Fitness_Cal(sample,P,stdData,gamma)
        P_fitness = fitness_update(P_Summary,P,P_fitness,PreStd,gamma,stdData)
        print('Processing Data Chunk '+str(t))
        if t == 0:
            P = P
            PF = np.asarray(P_fitness)
        else:
            P,P_fitness = CE_Online(sample,P_Summary,P,P_fitness,stdData,gamma,PreStd)
            PF = np.asarray(P_fitness)
        P_Summary = ClusterSummary(P,PF,P_Summary,sample)
        PreStd,PFS = StoreInf(PF,PFS,PreStd,stdData)
    # Clustering procedure finishes
    [MinDist,ClusterIndice] = Cluster_Assign(AccSample,P)

170508288/170498071 [==============================] - 2s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:473: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Processing Data Chunk 0
Processing Data Chunk 2
Processing Data Chunk 3
Processing Data Chunk 4
Processing Data Chunk 5
Processing Data Chunk 7
Processing Data Chunk 8
Processing Data Chunk 13
Processing Data Chunk 15
Processing Data Chunk 16
Processing Data Chunk 18
Processing Data Chunk 19
Processing Data Chunk 20
Processing Data Chunk 21
Processing Data Chunk 23
Processing Data Chunk 27
Processing Data Chunk 29
Processing Data Chunk 30
Processing Data Chunk 31
Processing Data Chunk 40
Processing Data Chunk 43
Processing Data Chunk 48


**Active Learning Phase**

In [ ]:
# Start Active Learning
sample_size = np.shape(AccSample)[0]
num_S = round(0.4 * sample_size)
cluster_radius = compute_radius(MinDist, ClusterIndice)
delta = np.mean(cluster_radius)
sigma = np.std(cluster_radius)
dense_idx = np.where(cluster_radius <= abs(delta - sigma))[0]
sparese_idx = np.where(cluster_radius > abs(delta - sigma))[0]

FetchIndex = []
UnlabeledIndex = []
InterDist = squareform(pdist(P))

FetchIndex = []
UnlabeledIndex = []
InterDist = squareform(pdist(P))

for i in range(np.shape(P)[0]):
  tempcluster = np.where(ClusterIndice == (i))
  d1 = []
  if i in dense_idx:
    for j in range(len(tempcluster[0])):
      d1.append(np.linalg.norm(AccSample[tempcluster[0][j], :] - P[i, :]))
    fetchSize = round(num_S * cluster_radius[i] / (np.sum(cluster_radius)))
    fetchSize = num_S * len(d1) / np.shape(AccSample)[0]
    sortIndex1 = np.argsort(d1)
    fet1 = tempcluster[0][sortIndex1[:int(round(fetchSize * 1))]]
    fet1 = fet1.astype(int)
    fet2 = []
    FetchIndex = np.append(FetchIndex, fet1)
    FetchIndex = np.append(FetchIndex, fet2)
    UnlabeledIndex = np.append(UnlabeledIndex,tempcluster[0][sortIndex1[int(round(fetchSize * 1)):round(len(d1))]])
  else:
      temp_interdist = InterDist[i, :]
      temp_rank = np.argsort(temp_interdist)
      temp_neigh1 = P[temp_rank[0], :]
      temp_neigh2 = P[temp_rank[1], :]
      d1 = []
      for j in range(len(tempcluster[0])):
        d1.append(np.linalg.norm(AccSample[tempcluster[0][j], :] - P[i, :]))
      # fetchSize = round(num_S * cluster_radius[i] / np.sum(cluster_radius))
      fetchSize = num_S * len(d1) / np.shape(AccSample)[0]
      sortIndex1 = np.argsort(d1)
      fet1 = tempcluster[0][sortIndex1[:int(round(fetchSize * 0.5))]]
      fet1 = fet1.astype(int)
      fil_index = sortIndex1[-int(round(len(d1) / 2)):]
      d2 = []
      for k in range(len(fil_index)):
          temp_d1 = np.linalg.norm(AccSample[tempcluster[0][fil_index[k]], :] - temp_neigh1)
          temp_d2 = np.linalg.norm(AccSample[tempcluster[0][fil_index[k]], :] - temp_neigh2)
          temp_ratio1 = max(temp_d1, temp_d2) / (min(temp_d1, temp_d2))
          temp_ratio2 = (temp_d1 + temp_d2) / (np.linalg.norm(temp_neigh1 - temp_neigh2))
          d2.append(temp_ratio1)
      sortIndex2 = np.argsort(d2)
      candidate_fet2 = fil_index[sortIndex2[:int(round(fetchSize * 0.8))]]
      sum_dist = []
      for ii in range(len(candidate_fet2)):
        candidate_d1 = np.linalg.norm(AccSample[tempcluster[0][candidate_fet2[ii]], :] - temp_neigh1)
        candidate_d2 = np.linalg.norm(AccSample[tempcluster[0][candidate_fet2[ii]], :] - temp_neigh1)
        sum_dist.append(candidate_d1 + candidate_d2)

      sortIndex3 = np.argsort(sum_dist)
      fet2 = tempcluster[0][candidate_fet2[sortIndex3[:int(round(fetchSize * 0.5))]]]
      fet2 = fet2.astype(int)
      FetchIndex = np.append(FetchIndex, fet1)
      FetchIndex = np.append(FetchIndex, fet2)
      UnlabeledIndex = np.append(UnlabeledIndex, tempcluster[0][sortIndex1[int(round(fetchSize * 0.5)):round(len(d1) / 2)]])
      UnlabeledIndex = np.append(UnlabeledIndex, tempcluster[0][fil_index[sortIndex2[int(round(fetchSize * 0.5)):len(d2)]]])
FetchIndex = FetchIndex.astype(int)
UnlabeledIndex = UnlabeledIndex.astype(int)

In [ ]:
queried_data = AccSample[FetchIndex][:]
queried_label1 = label[FetchIndex]
print("Number of clusters: ",np.shape(P))
print("Number of queried labels: ",len(label[FetchIndex]))
# Build CNN using active learned samples
[nq, lq] = np.shape(queried_data)
nt = X_test.shape[0]
queried_data1 = np.reshape(queried_data, (nq,32,32,3))
queried_label1 = np.reshape(queried_label1,(nq, 1))
testing_data1 = np.reshape(X_test, (nt,32,32,3))
testing_label1 = np.reshape(y_test, (nt,1))
queried_label1 = to_categorical(queried_label1, num_classes=10)
testing_label1 = to_categorical(testing_label1, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
y_train = to_categorical(y_train, num_classes=10)

Number of clusters:  (40, 3072)
Number of queried labels:  20005


**Performance Evaluation using the regular CNN model**

In [ ]:
# model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
#     Conv2D(32, (3,3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Dropout(0.3),
#     Conv2D(64, (3,3), activation='relu'),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Dropout(0.3),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.3),
#     Dense(10, activation='softmax')
# ])
# print("labels of y_train are", np.unique(y_train[:]))
# print("labels of y_test are", np.unique(y_test[:]))

# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# model.fit(queried_data1, queried_label1, epochs=20)
# loss, acc = model.evaluate(X_test, y_test)
# print("Model accuracy on test data is: {:6.3f}%".format(100 * acc))
############Model performance using the entire training set ##########
# model2 = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
#     Conv2D(32, (3,3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Dropout(0.3),
#     Conv2D(64, (3,3), activation='relu'),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Dropout(0.3),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.3),
#     Dense(10, activation='softmax')
# ])
# print("labels of y_train are", np.unique(y_train[:]))
# print("labels of y_test are", np.unique(y_test[:]))

# model2.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# model2.fit(X_train, y_train, epochs=20)
# loss2, acc2 = model2.evaluate(X_test, y_test)
# print("Model accuracy on test data without active learning  is: {:6.3f}%".format(100 * acc2))

**Performance Evaluation using Vgg16 Architecture**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16        import VGG16
from tensorflow.keras.optimizers import SGD
def create_model_vgg16():
    tf_input = Input(shape=(32, 32, 3))
    model = VGG16(input_tensor=tf_input, include_top=False)
    output_pooled1 = Flatten()(model.output)
    output_pooled2 = Dense(512, activation='relu')(output_pooled1)
    output_pooled = Dense(10, activation='softmax')(output_pooled2)
    return Model(model.input, output_pooled)
Vggmodel = create_model_vgg16()
# Vggmodel.summary()
opt = SGD(lr=0.001, momentum=0.9)
Vggmodel.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history = Vggmodel.fit(queried_data1, queried_label1, epochs=10)
loss, score = Vggmodel.evaluate(X_test, y_test, verbose=0)
print("Model accuracy on test data is: {:6.3f}%".format(100 * score))

58900480/58889256 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
626/626 [==============================] - 48s 61ms/step - loss: 1.1040 - accuracy: 0.6119
Epoch 2/10
626/626 [==============================] - 37s 60ms/step - loss: 0.6695 - accuracy: 0.7736
Epoch 3/10
626/626 [==============================] - 37s 60ms/step - loss: 0.5164 - accuracy: 0.8238
Epoch 4/10
626/626 [==============================] - 37s 59ms/step - loss: 0.4031 - accuracy: 0.8616
Epoch 5/10
626/626 [==============================] - 37s 60ms/step - loss: 0.3196 - accuracy: 0.8897
Epoch 6/10
626/626 [==============================] - 37s 59ms/step - loss: 0.2466 - accuracy: 0.9158
Epoch 7/10
626/626 [==============================] - 37s 59ms/step - loss: 0.1743 - accuracy: 0.9398
Epoch 8/10
626/626 [==============================] - 38s 60ms/step - loss: 0.1260 - accuracy: 0.9574
Epoch 9/10
626/626 [==============================] - 38s 61ms/step - loss: 0.1051 - accuracy: 0.9640
Epoch 10/10
626/626 [==============================] - 38s 60ms/step - loss: 0.071

**VGG16 Performance using the entire training set**

In [ ]:
Vggmodel2 = create_model_vgg16()
# Vggmodel.summary()
opt = SGD(lr=0.001, momentum=0.9)
Vggmodel2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history2 = Vggmodel2.fit(X_train, y_train, epochs=10)
loss2,score2 = Vggmodel2.evaluate(X_test, y_test, verbose=0)
print("Model accuracy on test data is: {:6.3f}%".format(100 * score2))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
1563/1563 [==============================] - 96s 61ms/step - loss: 0.8652 - accuracy: 0.6970
Epoch 2/10
1563/1563 [==============================] - 94s 60ms/step - loss: 0.5141 - accuracy: 0.8214
Epoch 3/10
1563/1563 [==============================] - 94s 60ms/step - loss: 0.3888 - accuracy: 0.8649
Epoch 4/10
1563/1563 [==============================] - 93s 60ms/step - loss: 0.3025 - accuracy: 0.8931
Epoch 5/10
1563/1563 [==============================] - 93s 60ms/step - loss: 0.2259 - accuracy: 0.9222
Epoch 6/10
1563/1563 [==============================] - 94s 60ms/step - loss: 0.1674 - accuracy: 0.9428
Epoch 7/10
1563/1563 [==============================] - 94s 60ms/step - loss: 0.1252 - accuracy: 0.9570
Epoch 8/10
1563/1563 [==============================] - 94s 60ms/step - loss: 0.0926 - accuracy: 0.9688
Epoch 9/10
1563/1563 [==============================] - 94s 60ms/step - loss: 0.0737 - accuracy: 0.9744
Epoch 10/10
1563/1563 [==============================] - 94s 60m

In [ ]:
import tensorflow as tf